PIP install function:

In [1]:
%pip install requests
%pip install python-dotenv
%pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Imports libraries & load API Key

In [1]:
from OpenAI import OpenAI
from dotenv import load_dotenv
import os

#Environment variables load from the .env file.
load_dotenv("sjapi.env")

#Print value of the OPENAI_API_KEY variable.
print(os.getenv("OPENAI_API_KEY"))

sk-sarah-v1zQsnpdM7XoXOawJmrJT3BlbkFJ4hLCy5TU2r5UrHJQniBj


Set up API client & create message list

In [3]:
#api_key definition to connect to Open API.
api_key = os.getenv("OPENAI_API_KEY")

#Connection to OpenAI API with API key.
openai.api_key = api_key

#Prompt for the user to enter a message.
prompt = input("Please enter a message: ")

#List of messages to send ot the API. 
message_list = [
    #The system message. Details can be added about how the model should respond.
    {
        "role": "system",
        "content": "You're a very helpful Assistant. Thank you."
    },
    #The user message with the prompt that the model will respond to. 
    {
        "role": "user",
        "content": prompt
    }
]

'''List of Tools'''

# Define the tools with their schemas
tools = [
    { 
        "type": "function",
        "function": {
            "name": "fetch_data_from_url",
            "description": "Fetches data from a specified URL",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The URL to fetch data from"
                    },
                },
                "required": ["url"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_factorial",
            "description": "Calculates the factorial of a number",
            "parameters": {
                "type": "object",
                "properties": {
                    "n": {
                    "type": "integer",
                    "description": "The number to calculate the factorial of"
                    },
                },
            "required": ["n"]
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the contents of a file",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "The path to the file to read"
                },
            },
            "required": ["file_path"]
            },
        },
    },
]

'''Call the API'''
#This can make a chat completions API call to OpenAI.
#'response' contains the model's response to the prompt.
response = openai.chat.completions.create(
    model='gpt-3.5-turbo', #model name
    messages=message_list, #messages list
    tool_choice="auto",
    tools=tools
)

#Prints the whole response object with all teh details.
print(response)

#Navigate through the response object to get model's response & tool tokens API call used.
print(f'Response:\n\n{response.choices[0].message.content}\n\n')
print(f'Total tokens used: {response.usage.total_tokens}')

ChatCompletion(id='chatcmpl-9kNGaoSOdpKM7dv3uwUqjI9eTly3S', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1720839552, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=133, total_tokens=143))
Response:

Hello! How can I assist you today?


Total tokens used: 143


Created functions & list of function descriptions.

In [4]:
#Fetching Data from a URL.
def fetch_data_from_url(url):
    response = requests.get(url)
    return f"Data from {url}: {response.status_code}"

#Example URL
url = "https://platform.openai.com/playground/chat?models=gpt-3.5-turbo-1106"

#Calculating Factorial.
def calculate_factorial(n):
    if n == 0 or n == 1:
        return 1
    else:
        result = 1
        for i in range(2, n + 1):
            result *= i  
        return f"Factorial of {n}: {result}"

#Reading a File.
def read_file(file_path):
    with open(file_path, 'r') as file:
        data = file.read()
    #Return first 50 characters for brevity
    return f"Contents of {file_path}: {data[:50]}..."

file_path = "sjapi.env"

#Functions list to call with arguments

functions_to_call = [
    partial(fetch_data_from_url, "https://platform.openai.com/playground/chat?models=gpt-3.5-turbo-1106"),
    partial(calculate_factorial, 5),
    partial(read_file, "sjapi.env")
    ]

# Function to call other functions in parallel
def call_functions_in_parallel(functions):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Start the operations and mark each future with its function
        future_to_function = {executor.submit(fn): fn for fn in functions}
        results = []
        for future in concurrent.futures.as_completed(future_to_function):
            fn = future_to_function[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as exc:
                results.append(f"{fn.__name__} generated an exception: {exc}")
    return results

# Call the functions in parallel
results = call_functions_in_parallel(functions_to_call)
print(results)

['Factorial of 5: 120', 'Contents of sjapi.env: OPENAI_API_KEY = "sk-sarah-v1zQsnpdM7XoXOawJmrJT3B...', 'Data from https://platform.openai.com/playground/chat?models=gpt-3.5-turbo-1106: 403']
